In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb
import copy

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./DyHead/atss_r50_fpn_dyhead_1x_coco.py')
root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = './stratified_kfold/cv_train_1.json' 
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = './stratified_kfold/cv_val_1.json'
# cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 2

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/atss_swin_t_fpn_dyhead_1x'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.runner.max_epochs = 100
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [4]:
# build_dataset
# datasets = [build_dataset(cfg.data.train),build_dataset(cfg.data.val)]
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


[
 CocoDataset Train dataset with number of images 3928, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3154  | 1 [Paper]     | 5076  | 2 [Paper pack]  | 741   | 3 [Metal]   | 780   | 4 [Glass]    | 752   |
 | 5 [Plastic]       | 2331  | 6 [Styrofoam] | 1028  | 7 [Plastic bag] | 4125  | 8 [Battery] | 121   | 9 [Clothing] | 396   |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [6]:
# 모델 학습
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


2022-03-28 15:20:26,397 - mmdet - INFO - Start running, host: root@6f46492e33de, work_dir: /opt/ml/CV_18_ysw/Mmdetection/_boost_/work_dirs/atss_swin_t_fpn_dyhead_1x
2022-03-28 15:20:26,399 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(LOW         ) IterTimerHook          

/opt/ml/detection/baseline/mmdetection/mmdet/models/dense_heads/atss_head.py:196: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  & (labels < bg_class_ind)).nonzero().squeeze(1)
2022-03-28 15:22:04,297 - mmdet - INFO - Epoch [1][50/1964]	lr: 4.995e-06, eta: 1 day, 11:30:22, time: 1.809, data_time: 0.059, memory: 21208, loss_cls: 1.1080, loss_bbox: 0.9631, loss_centerness: 0.6646, loss: 2.7357, grad_norm: 5.7843
2022-03-28 15:23:32,067 - mmdet - INFO - Epoch [1][100/1964]	lr: 9.990e-06, eta: 1 day, 10:57:14, time: 1.755, data_time: 0.009, memory: 21208, loss_cls: 0.9972, loss_bbox: 0.9200, loss_centerness: 0.6216, loss: 2.5388, grad_norm: 4.8354
2022-03-28 15:25:07,494 - mmdet - INFO - Epoch [1][150/1964]	lr: 1.499e-05, eta: 1 day, 11:45:15, time: 1.909, data_time: 0.015

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 955/955, 3.4 task/s, elapsed: 279s, ETA:     0s

2022-03-29 03:19:38,571 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.71s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=9.73s).
Accumulating evaluation results...


2022-03-29 03:19:55,222 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.118
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.097
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.462
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.462
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.462
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.133
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=5.40s).


2022-03-29 03:20:05,332 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_12.pth.
2022-03-29 03:20:05,334 - mmdet - INFO - Best bbox_mAP is 0.0800 at 12 epoch.
2022-03-29 03:20:05,340 - mmdet - INFO - Epoch(val) [12][955]	bbox_mAP: 0.0800, bbox_mAP_50: 0.1180, bbox_mAP_75: 0.0840, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0140, bbox_mAP_l: 0.0970, bbox_mAP_copypaste: 0.080 0.118 0.084 0.000 0.014 0.097
2022-03-29 03:21:38,520 - mmdet - INFO - Epoch [13][50/1964]	lr: 1.704e-06, eta: 23:33:03, time: 1.863, data_time: 0.066, memory: 21208, loss_cls: 0.5517, loss_bbox: 0.5774, loss_centerness: 0.6126, loss: 1.7418, grad_norm: 4.1251
2022-03-29 03:23:05,345 - mmdet - INFO - Epoch [13][100/1964]	lr: 1.704e-06, eta: 23:31:27, time: 1.736, data_time: 0.011, memory: 21208, loss_cls: 0.5506, loss_bbox: 0.5509, loss_centerness: 0.6110, loss: 1.7125, grad_norm: 7.7177
2022-03-29 03:24:37,390 - mmdet - INFO - Epoch [13][150/1964]	lr: 1.704e-06, eta: 23:30:01, time: 1.841, data_time: 0.01

KeyboardInterrupt: 

In [ ]:
import torch
torch.cuda.empty_cache()